In [1]:
# imports
from pathlib import Path
from vgio.quake.bsp import Bsp
import pyvista as pv
import numpy as np
from dotdict import dotdict
from collections import defaultdict
import math

In [2]:
# BSP loading
quake_root = Path("D:\\Games\\quake")

content_types = {}
content_types[-1]=dotdict({"contents":-1, "type":"CONTENTS_EMPTY"})
content_types[-2]=dotdict({"contents":-2, "type":"CONTENTS_SOLID"})
content_types[-3]=dotdict({"contents":-3, "type":"CONTENTS_WATER"})
content_types[-4]=dotdict({"contents":-4, "type":"CONTENTS_SLIME"})
content_types[-5]=dotdict({"contents":-5, "type":"CONTENTS_LAVA"})
content_types[-6]=dotdict({"contents":-6, "type":"CONTENTS_SKY"})

def read_bsp(name):
   with Bsp.open(f"D:\\Games\\quake\\id1\\maps\\{name}.bsp") as bsp_file:
      planes = bsp_file.planes
      meshes = bsp_file.meshes()            
      clipnodes = bsp_file.clip_nodes

      # main model
      model = bsp_file.models[0]

      hulls = []
      for node in clipnodes:
         hulls.append(dotdict({
            "plane": planes[node.plane_number],
            "children": [node.children[0], node.children[1]]
         }))
      for node in hulls:
         def attach_node(side):
            children = node.children
            id = children[side] 
            if id<0:
               children[side] = content_types[id]
            else:
               children[side] = hulls[id]
         attach_node(0)
         attach_node(1)
      # display 32 unit bsp
      def dump_hull(node):
         content = node.get("content", None)
         if content:
            print(content)
         else:
            children = node.get("children")
            dump_hull(children[0])
            dump_hull(children[1])
      # dump_hull(hulls[bsp_file.models[0].head_node[1]])

      faces = bsp_file.faces[model.first_face:model.first_face + model.number_of_faces]
      tris = []
      for face in faces:
         edges = bsp_file.surf_edges[face.first_edge:face.first_edge + face.number_of_edges]
         verts = []
         for edge_id in edges:
            v = bsp_file.edges[abs(edge_id)].vertexes
            v0,v1 = v if edge_id<0 else reversed(v)
            verts.append(v0)
         # Ignore degenerate faces
         if len(verts) < 3:
            continue
            
         verts.insert(0,len(verts))         
         tris.append(verts)
         
      return pv.PolyData(np.array([[v.x,v.y,v.z] for v in bsp_file.vertexes]), np.hstack(tris)), model, hulls

In [15]:
# get BSP data
pdata, model, hulls = read_bsp("aerowalk")

# bounding box
bmin = model.bounding_box_min
bmax = model.bounding_box_max

xmin = int(bmin[0])
xmax = int(math.ceil(bmax[0]))
ymin = int(bmin[1])
ymax = int(math.ceil(bmax[1]))

precision = 8

xrange = (xmax - xmin + 1) // precision
yrange = (ymax - ymin + 1) // precision

print(f"map extents: {xmin}/{ymin} - {xmax}/{ymax}")

def world_to_id(x,y):
    return int(x // precision) + yrange * int(y // precision)

map extents: -991/-959 - 415/727


In [24]:
# pv.set_jupyter_backend('ipyvtklink')  
pv.set_jupyter_backend('pythreejs')  
# pv.set_jupyter_backend('static')  

# custom clamp
clamp = lambda x, l, u: l if x < l else u if x > u else x

def lerp(a,b,t):
   return a+(b-a)*t

def v_lerp(a,b,t):
   return [
      lerp(a[0],b[0],t),
      lerp(a[1],b[1],t),
      lerp(a[2],b[2],t)]

def plane_dot(plane,p):
   n = plane.normal
   return (n[0]*p[0] + n[1]*p[1] + n[2]*p[2], plane.distance)

def classify_point(plane, p):
   n = plane.normal
   return int(n[0]*p[0] + n[1]*p[1] + n[2]*p[2] < plane.distance)

def bsp_locate(node,pos):
   tabs = 0
   while not node.contents:
      n = node.plane.normal
      side = classify_point(node.plane,pos)
      # print(f"{' '*tabs}{n[0]} {n[1]} {n[2]} > {node.plane.distance}: {side}")
      node = node.children[side]
   return node

def bsp_ray_collect(node,p0,p1,t0,t1,out):
   if node.contents: return

   dist,node_dist=plane_dot(node.plane,p0)
   otherdist,_=plane_dot(node.plane,p1)
   side,otherside=int(dist<node_dist),int(otherdist<node_dist)
   if side==otherside:
      # go down this side
      return bsp_ray_collect(node.children[side],p0,p1,t0,t1,out)

   # crossing a node
   t=dist-node_dist
   if t<0:
      t=t+0.03125
   else:
      t=t-0.03125

   # cliping fraction
   frac=clamp(t/(dist-otherdist),0,1)
   tmid,pmid=lerp(t0,t1,frac),v_lerp(p0,p1,frac)

   is_solid = bsp_locate(node.children[side],pmid).contents!=-2
   is_otherside_solid = bsp_locate(node.children[otherside],pmid).contents!=-2
   if is_solid and not is_otherside_solid:
      scale=1 if side==0 else -1
      n=node.plane.normal
      out.append(dotdict({
         "n" : [scale*n[0],scale*n[1],scale*n[2]],
         "t" : tmid,
         "pos": pmid
      }))

   if is_otherside_solid and not is_solid:
      scale=1 if otherside==0 else -1
      n=node.plane.normal
      out.append(dotdict({
         "n" : [scale*n[0],scale*n[1],scale*n[2]],
         "t" : tmid,
         "pos": pmid
      }))

   # go down both side
   bsp_ray_collect(node.children[side],p0,pmid,t0,tmid,out)
   bsp_ray_collect(node.children[int(not side)],pmid,p1,tmid,t1,out)

def bsp_ray_intersect(node,p0,p1,t0,t1,out):
    contents=node.contents  
    if contents:
        # is "solid" space (bsp)
        if contents!=-2:
            out.all_solid = False
            if contents==-1:
                out.in_open = True
            else:
                out.in_water = True
        else:
            out.start_solid = True
        # empty space
        return True

    dist,node_dist=plane_dot(node.plane,p0)
    otherdist,_=plane_dot(node.plane,p1)
    side,otherside=dist>node_dist,otherdist>node_dist
    if side==otherside:
        # go down this side
        return bsp_ray_intersect(node.children[0 if side else 1],p0,p1,t0,t1,out)

    # crossing a node
    t=dist-node_dist
    if t<0:
        t=t+0.03125
    else:
        t=t-0.03125

    # cliping fraction
    frac=clamp(t/(dist-otherdist),0,1)
    tmid,pmid=lerp(t0,t1,frac),v_lerp(p0,p1,frac)
    if not bsp_ray_intersect(node.children[0 if side else 1],p0,pmid,t0,tmid,out):
        return False

    child_id = 0 if not side else 1
    if bsp_locate(node.children[child_id],pmid).contents != -2:
        return bsp_ray_intersect(node.children[child_id],pmid,p1,tmid,t1,out)

    # never got out of the solid area
    if out.all_solid:
        return False

    scale=1 if side else -1
    n=node.plane.normal
    out.n = [scale*n[0],scale*n[1],scale*n[2]]
    out.t = tmid
    out.pos = pmid
    return False

pl = pv.Plotter()
# base geometry
pl.add_mesh(pdata, style='wireframe', lighting=False)

# stores intersection points in "column"
voxel_grid = defaultdict(list)
# head_node 0 references *nodes* array, not clipnodes
root_node = hulls[model.head_node[1]]
for x in range(xmin, xmax, precision):
   for y in range(ymin, ymax, precision):
      out = []
      bsp_ray_collect(root_node,[x,y,bmin[2]],[x,y,bmax[2]],0,1,out)
      for hit in out:
         # walkable?
         if hit.n[2]>0.7:
            voxel_grid[world_to_id(x,y)].append(hit)

# sort all entries by "height"
for k,voxels in voxel_grid.items():
   voxels.sort(key=lambda v: v.pos[2])

# get "slices" (starting from bottom)
level = 0
while True:   
   points = [voxels[level] for k,voxels in voxel_grid.items() if len(voxels)>level]
   if len(points)==0: break
   # "flatten" 16 unit gap (stairs) to fall under the "merge" criteria
   surf = pv.wrap(np.array([(p.pos[0],p.pos[1],p.pos[2]/2) for p in points])).delaunay_2d(alpha=8.0) # .decimate_pro(0.9, preserve_topology=True)
   color = f"#0000{int(((level+1)*16)%256):02X}"
   pl.add_mesh(surf.scale((1,1,2)), color=color, opacity=0.5)
   edges = surf.extract_feature_edges()
   pl.add_mesh(edges, color=color)
   level += 1

pl.show()


c:\Users\Frederic\Source\Repos\love-quake\.env\lib\site-packages\pyvista\core\pointset.py:262: PyVistaDeprecationWarning: You did not specify a value for `inplace` and the default value will be changing to `False` in future versions for point-based meshes (e.g., `PolyData`). Please make sure you are not assuming this to be an inplace operation.
  warnings.warn(DEFAULT_INPLACE_WARNING, PyVistaDeprecationWarning)


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_f7d01737-f49e-440e-9580-6a47bca7a350'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_8fc18954-7b4f-4eca-9135-ac685b8d9e80'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_d06c65fc-ee41-481c-855d-45b9694add28'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_6b5a248d-4c79-4192-b565-bf2525831859'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_2cabed84-c024-490a-97e0-a65082a52dc8'.

In [ ]:

# Define a simple linear surface
x = np.array([0,1,2,3,4,5,6,7,8,9])
y = np.array([0,1,2,3,4,5,6,7,8,9])
x, y = np.meshgrid(x, y)
# Create and plot structured grid
z = []
for y in range(10):
    points = []
    for x in range(10):
        if abs(x-5)<2:
            points.append(4)
        else:
            points.append(0)            
    z.append(points)

grid = pv.StructuredGrid(x, y, np.array(z))

# mesh = grid.contour(scalars=grid.points[:, -1], method='marching_cubes')

pl = pv.Plotter()
# pl.add_mesh(mesh, color=True, show_edges=True)
pl.add_mesh(grid, style='points', point_size=10.0, render_points_as_spheres=True, lighting=False)
pl.show()

TypeError: Invalid parameters. Expecting one of the following:
 - No arguments
 - Filename as the only argument
 - StructuredGrid as the only argument
 - Single `numpy.ndarray` as the only argument - Three `numpy.ndarray` as the first three arguments

In [ ]:
   
levels=[]
for h,points in heights.items():
   if len(points)>2:
      grid = {}
      # place points on 2d grid
      for p in points:
         id = int(p[0] - xmin)//8 + yrange * (int(p[1] - ymin) // 8)
         grid[id] = p
      
      # scan grid to find boundaries
      spans = defaultdict(list)
      for x in range(xrange):
         count = 0
         y = 0
         # find first cell
         while y<yrange:
            id0 = x + yrange * y
            if id0 in grid:
               # collect 1st point
               p0 = grid[id0]
               # search 2nd point
               y += 1
               while y<yrange:
                  id1 = x + yrange * y
                  # gap?
                  if id1 not in grid:
                     p1 = grid[id0]
                     # avoid degenerate case (single point)
                     # if p1!=p0:
                     spans[x].append(p0)
                     spans[x].append(p1)
                     count += 2
                     break
                  id0 = id1
                  y += 1
            else:
               y += 1
         # must have an even number of points
         assert(count%2==0)
      # find holes
      holes = []
      points = []
      for x,span in spans.items():                  
         for i,p in enumerate(span):
            points.append(p)
            if i>0 and i%2==0:
               holes.append(span[i-1])
               holes.append(p)
      
      level = pv.wrap(np.array(points))
      levels.append(level)
      # pl.add_mesh(level, style='points', point_size=10.0, render_points_as_spheres=True, lighting=False)
      surf = level.contour([1], values, method='marching_cubes')
      pl.add_mesh(surf, color=True, show_edges=True)
      # if len(holes)>0:
      #    #pl.add_mesh(pv.wrap(np.array(holes)), style='points', point_size=10.0, render_points_as_spheres=True, lighting=False)
      #    mask = pv.wrap(np.array(holes))
      #    surf = level.delaunay_2d(edge_source=surf_mask)
      #    pl.add_mesh(surf, color=True, show_edges=True)
      # else:
      #    surf = level.delaunay_2d(alpha=1.0)
      #    pl.add_mesh(surf, color=True, show_edges=True)

def height(v):
   return np.floor(v[2])

In [ ]:
import pyvista as pv
import numpy as np

pv.set_jupyter_backend('ipyvtklink')  

# Define a simple linear surface
plotter = pv.Plotter()
z = []
edges = []
points = {}
for width in [10,20]:
    for y in range(10):
        y += width
        for x in range(width):
            if abs(x-width/2)>2 or abs(y-5-width)>2:                
                e0 = (float(8*x),float(8*y),2)
                if width==20 and x<5:
                    e0 = (float(8*x),float(8*y),2-3*x)
                points[int(e0[0]/8)+int(e0[1]/8)*64] = e0
                z.append(e0)

for y in range(5):
    edges.append(points[y+19*64])
    edges.append(points[y+20*64])

grid = pv.wrap(np.array(z))
assert(len(edges)>0)
edges = pv.wrap(np.array(edges))
surf = grid.delaunay_2d(alpha=8, edge_source=edges)

plotter.add_mesh(surf, color='#ffff00', style='wireframe')
plotter.add_mesh(grid, style='points', point_size=5.0, render_points_as_spheres=True, lighting=False)
plotter.add_mesh(edges, color='#ff0000', style='points', point_size=10.0, render_points_as_spheres=True, lighting=False)

plotter.show_axes_all()
plotter.show()


ValueError: Empty meshes cannot be plotted. Input mesh has zero points.